# 🌍 Global CO₂ Analysis & Future Projections

## Introduction
This notebook presents a comprehensive analysis of global CO₂ emissions from the **Miuul Data Science Bootcamp** team. Our goal is to go beyond simple statistics and investigate the **causes** of pollution, **correlations** with economic factors, and provide **machine learning-based forecasts** for the future.

**Key Objectives:**
1.  **Exploratory Data Analysis (EDA):** Visualize historical trends and country-specific profiles.
2.  **Driver Analysis:** Understand how GDP, Population, and Energy Consumption drive emissions.
3.  **Machine Learning:** Predict future emissions (2025-2028) using Multivariate Polynomial Regression.
4.  **Insights:** Provide actionable recommendations based on data.

**Data Source:** Our World in Data (OWID) CO₂ dataset.

---


## 1. Libraries and Configuration
We start by importing standard data science libraries.
*   `pandas` & `numpy` for data manipulation.
*   `seaborn` & `matplotlib` for visualization.
*   `sklearn` for our machine learning models (Linear Regression, Polynomial Features).



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Configure Pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)
pd.set_option("display.width", 500)

# Define consistent colors for countries
COUNTRY_COLORS = {
    "China": "#E74C3C",  
    "United States": "#3498DB",  
    "Germany": "#F1C40F",  
    "Russia": "#8E44AD",  
    "Turkey": "#E67E22",  
    "India": "#2ECC71",  
}


## 2. Loading the Data
We define a robust `load()` function that checks multiple common paths (`/kaggle/input`, local folders) to find the dataset. This ensures the notebook runs on both Kaggle and local machines without changing code.



In [ ]:
def load():
    """
    Loads the dataset. Checks typical Kaggle input paths and local directories.
    """
    possible_paths = [
        "/kaggle/input/owid-co2-data/owid-co2-data.csv",  # Typical Kaggle Path
        "Datasets/owid-co2-data.csv",                     # Local Project Path
        "owid-co2-data.csv"                               # Current Directory
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            print(f"Loading data from: {path}")
            return pd.read_csv(path)
            
    raise FileNotFoundError("Dataset not found! Please add 'owid-co2-data.csv' to the notebook.")

df = load()
print("Data Shape:", df.shape)
df.head()


## 3. Feature Selection
We define the list of features we will use for our analysis and modeling. This includes economic indicators (GDP), demographics (Population), and energy metrics.



In [ ]:
FEATURES = [
    "year",
    "gdp",
    "population",
    "primary_energy_consumption",
    "energy_per_capita",
    "co2_per_capita",
    "co2_per_gdp",
    "coal_co2",
    "oil_co2",
    "gas_co2",
    "cement_co2",
    "flaring_co2",
    "methane",
    "nitrous_oxide",
]


## 4. Data Cleaning (Visualization Ready)
The `clean_and_balance_data_for_eda` function prepares the data for **Visual Analysis**. 
*   **Why Interpolate?** Real-world data often has gaps (e.g., missing GDP for a few years). For smooth line charts, we use linear interpolation.
*   **Note:** We do *not* use this specific function for Model Training to avoid data leakage (using future data to fill past gaps). This is strictly for EDA.



In [ ]:
def clean_and_balance_data_for_eda(data: pd.DataFrame) -> pd.DataFrame:
    """
    Applies interpolation (both directions) for smooth visualization/EDA.
    """
    print("\n--- Data Quality Report (Before Cleaning) ---")
    print("Missing Values (%):")
    print(data[["co2", "population", "gdp"]].isnull().mean() * 100)

    # Sort to ensure time-series is correct
    data = data.sort_values(["country", "year"])
    
    cols_to_interpolate = list(set(FEATURES + ["co2"]))
    cols_to_interpolate = [c for c in cols_to_interpolate if c in data.columns]

    def fill_group(group):
        # We use 'both' limit direction to fill gaps at start and end for plotting
        group[cols_to_interpolate] = group[cols_to_interpolate].interpolate(
            method="linear", limit_direction="both"
        )
        return group

    data = data.groupby("country", group_keys=False).apply(fill_group)

    print("\n--- Data Quality Report (After Interpolation) ---")
    print("Missing Values (%):")
    print(data[["co2", "population", "gdp"]].isnull().mean() * 100)

    return data

df_eda = clean_and_balance_data_for_eda(df.copy())


## 5. Helper Functions for Machine Learning
Here we define specialized functions for our ML pipeline.
**Key Concept: Time-Safe Imputation**
When training a model to predict the future, we must not "peak" at the future. Standard interpolation uses future points to fill past gaps.
*   `_country_time_safe_impute_after_split`: Fills missing values in the Test set using *only* past data (forward fill), ensuring no leakage.
*   `_build_global_avg`: Aggregates country data to create a "Global Average" dataset.



In [ ]:
def _build_global_avg(data: pd.DataFrame) -> pd.DataFrame:
    cols = list(set(FEATURES + ["co2"]))
    cols = [c for c in cols if c in data.columns and c != "year"]
    df_subset = data.groupby("year")[cols].mean(numeric_only=True).reset_index()
    df_subset = df_subset.sort_values("year")
    return df_subset

def _country_time_safe_impute_after_split(
    train_df: pd.DataFrame, test_df: pd.DataFrame, cols: list[str]
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Imputes missing values without lookahead bias.
    Train: Interpolate (since we have the full history).
    Test: Forward Fill ONLY (using only past known data).
    """
    tr = train_df.sort_values(["country", "year"]).copy()
    te = test_df.sort_values(["country", "year"]).copy()

    tr[cols] = tr[cols].replace([np.inf, -np.inf], np.nan)
    te[cols] = te[cols].replace([np.inf, -np.inf], np.nan)

    # Train: Interpolate
    def fill_train(g: pd.DataFrame) -> pd.DataFrame:
        g[cols] = g[cols].interpolate(method="linear", limit_direction="both").ffill().bfill()
        return g

    tr = tr.groupby("country", group_keys=False).apply(fill_train)

    # Test: Ffill only (simulate real-time)
    te = te.groupby("country", group_keys=False).apply(lambda g: g.assign(**{c: g[c].ffill() for c in cols}))

    # If test starts with NaN, use the last value from Train
    last_vals = tr.groupby("country")[cols].last()
    te = te.set_index("country")
    for c in cols:
        te[c] = te[c].fillna(last_vals[c])
    te = te.reset_index()

    return tr, te

def _time_safe_impute_after_split(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    fill_cols: list[str],
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Train/Test split SONRASI imputasyon (Global Average icin).
    - Train: interpolate(both) + ffill/bfill
    - Test : ffill (sadece gecmis) + bastaki NaN -> train son degeri
    """
    tr = train_df.copy()
    te = test_df.copy()

    # inf guard
    tr[fill_cols] = tr[fill_cols].replace([np.inf, -np.inf], np.nan)
    te[fill_cols] = te[fill_cols].replace([np.inf, -np.inf], np.nan)

    tr[fill_cols] = tr[fill_cols].interpolate(method="linear", limit_direction="both").ffill().bfill()

    te[fill_cols] = te[fill_cols].ffill()
    for c in fill_cols:
        if te[c].isna().any():
            te[c] = te[c].fillna(tr[c].iloc[-1])

    return tr, te


## 6. Model Evaluation
We evaluate our **Multivariate Linear Regression** model.
*   **Split:** Train (2000-2018), Test (2019-2024).
*   **Metrics:** RMSE (Root Mean Squared Error), MAE (Mean Absolute Error), R2 Score.
*   **Feature Importance:** We inspect the model coefficients to see which factors (GDP, Energy, etc.) have the biggest impact on CO2.



In [ ]:
def evaluate_model_multivariate_time_safe(data: pd.DataFrame) -> dict:
    print("\n--- Model Evaluation (Multivariate Global, TIME-SAFE) ---")

    # 1) Split based on Years
    train_raw = data[(data["year"] >= 2000) & (data["year"] <= 2018)].copy()
    test_raw = data[(data["year"] >= 2019) & (data["year"] <= 2024)].copy()

    # 2) Impute Safely
    cols_for_country = [c for c in (FEATURES + ["co2"]) if c in data.columns and c not in ["year", "country"]]
    train_imp, test_imp = _country_time_safe_impute_after_split(train_raw, test_raw, cols=cols_for_country)

    # 3) Build Global Average for Modeling
    df_train = _build_global_avg(train_imp)
    df_test = _build_global_avg(test_imp)

    model_cols = [c for c in FEATURES if c in df_train.columns]
    
    df_train = df_train.dropna(subset=["co2"] + model_cols)
    df_test = df_test.dropna(subset=["co2"] + model_cols)

    X_train = df_train[model_cols]
    y_train = df_train["co2"]
    X_test = df_test[model_cols]
    y_test = df_test["co2"]

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
    mae = float(mean_absolute_error(y_test, y_pred))
    r2 = float(r2_score(y_test, y_pred))

    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")

    print("\nFeature Importance (Coefficients):")
    coef_df = pd.DataFrame({"Feature": model_cols, "Coefficient": model.coef_})
    coef_df["Abs_Coef"] = coef_df["Coefficient"].abs()
    print(coef_df.sort_values("Abs_Coef", ascending=False))

    return {"rmse": rmse, "r2": r2}

# Run Evaluation
metrics = evaluate_model_multivariate_time_safe(df)


## 7. Forecasting Logic
To predict CO2 for 2025-2028, we first need to predict the **features** (GDP, Population, etc.) for those years.
1.  **Forecast Features:** Use Polynomial Regression (Degree 2) to extend the trend of each feature into the future.
2.  **Predict CO2:** Feed these forecasted features into our trained Multivariate Model to get the CO2 prediction.



In [ ]:
def forecast_features(data: pd.DataFrame, future_years: np.ndarray) -> pd.DataFrame:
    """
    Forecasts future values for input features (GDP, Pop, etc.) using polynomial regression.
    """
    forecasts = {}
    feature_cols = [c for c in FEATURES if c != "year" and c in data.columns]
    future_years_reshaped = future_years.reshape(-1, 1)

    for col in feature_cols:
        df_feat = data[["year", col]].dropna()
        if len(df_feat) < 5:
            last_val = df_feat[col].iloc[-1] if not df_feat.empty else 0
            forecasts[col] = np.full(len(future_years), last_val)
            continue

        X_feat = df_feat[["year"]]
        y_feat = df_feat[col]

        poly_feat = PolynomialFeatures(degree=2)
        X_poly_feat = poly_feat.fit_transform(X_feat)

        model_feat = LinearRegression()
        model_feat.fit(X_poly_feat, y_feat)

        future_poly = poly_feat.transform(future_years_reshaped)
        forecasts[col] = model_feat.predict(future_poly)

    return pd.DataFrame(forecasts, index=future_years.flatten())


def predict_co2_multivariate(data: pd.DataFrame, country_name: str | None = None):
    """
    Trains model on full history (2000-2024) and predicts 2025-2028.
    Calculates 95% Confidence Intervals.
    """
    if country_name:
        df_subset = data[data["country"] == country_name].copy()
    else:
        cols = list(set(FEATURES + ["co2"]))
        if "year" in cols: cols.remove("year")
        df_subset = data.groupby("year")[cols].mean(numeric_only=True).reset_index()

    model_cols = [c for c in FEATURES if c in df_subset.columns]
    df_train = df_subset[(df_subset["year"] >= 2000) & (df_subset["year"] <= 2024)].dropna(subset=["co2"] + model_cols)

    if len(df_train) < 10:
        return None, None, None, None, None, None

    X = df_train[model_cols]
    y = df_train["co2"]

    model = LinearRegression()
    model.fit(X, y)

    future_years = np.arange(2025, 2029)
    future_features_df = forecast_features(df_subset, future_years)
    future_features_df["year"] = future_years
    X_future = future_features_df[model_cols]
    
    predictions = model.predict(X_future)

    # Confidence Interval calculation
    y_pred_train = model.predict(X)
    residuals = y - y_pred_train
    std_error = np.std(residuals)

    ci_lower, ci_upper = [], []
    for i in range(len(predictions)):
        margin = 1.96 * std_error * np.sqrt(i + 1)
        ci_lower.append(predictions[i] - margin)
        ci_upper.append(predictions[i] + margin)

    return df_train, future_years, predictions, model, np.array(ci_lower), np.array(ci_upper)


## 8. Analysis: Global Trends
**Question:** How have global emissions changed over time?
**Observation:** We plot the mean global CO2 emission.



In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_eda, x="year", y="co2", estimator="mean", errorbar=None, color="black")
plt.title("Average Global CO2 Emissions Over Years")
plt.ylabel("CO2 Emissions (Million Tonnes)")
plt.xlabel("Year")
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 9. Analysis: Country Comparisons
**Question:** Which major economies contribute most to emissions?
We focus on: **China, USA, Germany, Russia, Turkey, India**.



In [ ]:
countries = ["China", "United States", "Russia", "Turkey", "Germany", "India"]
df_countries = df_eda[df_eda["country"].isin(countries)]

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_countries, x="year", y="co2", hue="country", palette=COUNTRY_COLORS)
plt.title("CO2 Emissions by Country")
plt.ylabel("CO2 Emissions (Million Tonnes)")
plt.legend(title="Country")
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 10. Correlation Analysis
**Question:** What variables are most strongly related to CO2 emissions?
We look at the post-1990 era to see modern relationships.
*   **High Correlation:** Means as one variable increases (e.g., GDP), CO2 also tends to increase.



In [ ]:
cols_to_corr = ["co2", "gdp", "population", "energy_per_capita", "co2_per_capita", "methane", "nitrous_oxide"]
cols_to_corr = [c for c in cols_to_corr if c in df_eda.columns]
df_corr = df_eda[df_eda["year"] > 1990][cols_to_corr].dropna()

plt.figure(figsize=(10, 8))
sns.heatmap(df_corr.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix (Post-1990)")
plt.show()


## 11. Multivariate Predictions (2025-2028)
Now we apply our `predict_co2_multivariate` function to forecast the future.
We plot **Historical Data**, **Model Predictions**, and the **95% Confidence Interval**.



In [ ]:
# Global Forecast
df_train_global, future_years, pred_global, model_global, ci_lower_global, ci_upper_global = predict_co2_multivariate(df_eda)

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_train_global, x="year", y="co2", label="Historical (2000-2024)", color="black")
plt.plot(future_years, pred_global, color="red", linestyle="--", label="Prediction (2025-2028)")
plt.fill_between(
    future_years.flatten(), ci_lower_global, ci_upper_global, color="red", alpha=0.2, label="95% Confidence Interval"
)
plt.title("Global CO2 Emissions Forecast (Multivariate Model)")
plt.ylabel("CO2 Emissions")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.show()

# Country Forecasts
plt.figure(figsize=(14, 7))
for country in countries:
    df_train, future_years, preds, model, ci_lower, ci_upper = predict_co2_multivariate(df_eda, country)
    if preds is not None:
        color = COUNTRY_COLORS.get(country, "gray")
        plt.plot(df_train["year"], df_train["co2"], label=f"{country}", color=color, alpha=0.6)
        plt.plot(future_years, preds, linestyle="--", color=color, linewidth=2)
        plt.fill_between(future_years.flatten(), ci_lower, ci_upper, color=color, alpha=0.1)

plt.title("CO2 Emissions Forecast by Country (2025-2028)")
plt.ylabel("CO2 Emissions")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 12. Driver Analysis & Recommendations
For each country, we calculate the correlation between CO2 and key drivers to generate automated recommendations.
*   **High GDP Correlation:** Suggests "Green Growth" needed (decoupling).
*   **High Energy Correlation:** Suggests Renewable Energy needs.
*   **High Population Correlation:** Suggests Urban Planning needs.



In [ ]:
print("--- Driver Analysis & Recommendations ---")
for country in countries:
    country_data = df_eda[df_eda["country"] == country].dropna(subset=["co2", "gdp", "energy_per_capita", "population"])
    if len(country_data) > 10:
        corr = country_data[["co2", "gdp", "energy_per_capita", "population"]].corr()["co2"]
        print(f"\nReport for {country}:")
        
        if corr.get("gdp", 0) > 0.9:
            print("  -> Insight: Strong link between Economic Growth and CO2. Recommendation: Focus on Green Tech to decouple.")
        if corr.get("energy_per_capita", 0) > 0.9:
            print("  -> Insight: High energy dependency. Recommendation: Accelerate transition to renewables.")
        if corr.get("population", 0) > 0.9:
            print("  -> Insight: Population driven. Recommendation: Sustainable urban planning.")


## 13. CO2 Impact Analysis (Population Driven)
**Scenario:** What if **per capita emissions** remained frozen at today's levels? How much would total emissions rise purely due to **population growth**?
This helps isolate the impact of demographics vs. industrial efficiency.



In [ ]:
print("--- CO2 Impact Analysis (Population Driven) ---")
plt.figure(figsize=(12, 6))

future_years_pop = np.arange(2025, 2029)

for country in countries:
    country_data = df_eda[df_eda["country"] == country].dropna(subset=["co2", "population", "co2_per_capita"])
    if not country_data.empty:
        last_hist_year = country_data["year"].max()
        last_per_capita = country_data.loc[country_data["year"] == last_hist_year, "co2_per_capita"].values[0]

        X_pop = country_data[["year"]]
        y_pop = country_data["population"]
        poly_pop = PolynomialFeatures(degree=2)
        X_poly_pop = poly_pop.fit_transform(X_pop)
        
        model_pop = LinearRegression()
        model_pop.fit(X_poly_pop, y_pop)

        future_years_reshaped = future_years_pop.reshape(-1, 1)
        future_poly_pop = poly_pop.transform(future_years_reshaped)
        pred_pop = model_pop.predict(future_poly_pop)

        # Baseline: If per capita emissions stay constant, what is the total CO2?
        pop_driven_co2 = pred_pop * last_per_capita
        
        color = COUNTRY_COLORS.get(country, "gray")
        plt.plot(future_years_pop, pop_driven_co2, linestyle=":", linewidth=2, color=color, label=f"{country} (Pop. Driven)")

plt.title("Projected CO2 if Per Capita Emissions Remain Constant (2025-2028)")
plt.ylabel("CO2 Emissions")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 14. Economic Carbon Intensity (CO2 per GDP)
**Metric:** How much CO2 do we emit for every dollar of GDP created?
*   **Declining trend:** Good! Means the economy is becoming cleaner/more efficient.
*   **Rising trend:** Bad. Growth is fueled by dirty energy.



In [ ]:
print("--- Carbon Intensity Analysis (CO2 per GDP) ---")
plt.figure(figsize=(12, 7))
for country in countries:
    country_data = df_eda[df_eda["country"] == country].sort_values("year")
    country_data = country_data[country_data["year"] >= 2000]
    if "co2_per_gdp" in country_data.columns:
        sns.lineplot(
            data=country_data,
            x="year",
            y="co2_per_gdp",
            label=country,
            color=COUNTRY_COLORS.get(country),
            linewidth=2,
        )
plt.title("Economic Carbon Intensity (CO2 per GDP) Trend (2000-2024)")
plt.ylabel("CO2 per GDP (kg per $)")
plt.xlabel("Year")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 15. Reduction Scenarios (Pathway to 2050)
If countries want to **halve** their emissions by 2050, how much do they need to reduce annually starting now?



In [ ]:
target_year = 2050
current_year = 2024
years_remaining = target_year - current_year

print(f"--- Required Annual Reduction to Halve Emissions by {target_year} ---")
for country in countries:
    country_data = df_eda[(df_eda["country"] == country) & (df_eda["year"] == current_year)]
    if not country_data.empty:
        current_co2 = country_data["co2"].values[0]
        target_co2 = current_co2 * 0.5
        # Compound annual reduction rate formula
        required_reduction = (1 - (target_co2 / current_co2) ** (1 / years_remaining)) * 100
        print(f"{country}: {required_reduction:.2f}% reduction per year needed.")


## 16. CO2 Per Capita
**Interpretation:** Total emissions don't tell the whole story. Per capita emissions show the intensity of lifestyle/industry per person.


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_countries, x="year", y="co2_per_capita", hue="country", palette=COUNTRY_COLORS)
plt.title("CO2 Emissions per Capita by Country")
plt.ylabel("CO2 per Capita (Tonnes)")
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 17. Population vs CO2 Growth (Indexed)
We define 2004 as the base year (Index=100) to compare the *rate* of growth between Population and CO2.
*   If CO2 line is above Population line -> Emissions growing faster than people (Industrialization).
*   If CO2 line is below -> Efficiency is improving.



In [ ]:
start_year_growth = 2004
end_year_growth = 2024

for country in countries:
    country_data = df_eda[
        (df_eda["country"] == country) & (df_eda["year"] >= start_year_growth) & (df_eda["year"] <= end_year_growth)
    ].sort_values("year")

    if not country_data.empty:
        base_pop = country_data["population"].iloc[0]
        base_co2 = country_data["co2"].iloc[0]

        if base_pop > 0 and base_co2 > 0:
            country_data["pop_index"] = (country_data["population"] / base_pop) * 100
            country_data["co2_index"] = (country_data["co2"] / base_co2) * 100

            fig, ax1 = plt.subplots(figsize=(10, 4))
            color_co2 = COUNTRY_COLORS.get(country, "tab:red")
            
            ax1.set_xlabel("Year")
            ax1.set_ylabel("CO2 Index (2004=100)", color=color_co2)
            ax1.plot(country_data["year"], country_data["co2_index"], color=color_co2, label="CO2 Growth", linewidth=2)
            ax1.tick_params(axis="y", labelcolor=color_co2)

            ax2 = ax1.twinx()
            ax2.set_ylabel("Population Index (2004=100)", color="black")
            ax2.plot(country_data["year"], country_data["pop_index"], color="black", linestyle="--", label="Population Growth")
            
            plt.title(f"{country}: Relative Growth (2004-2024)")
            plt.grid(True, linestyle="--", alpha=0.5)
            plt.show()


## 18. Population vs. CO2 per Capita
**Question:** Do larger populations correspond to higher *per capita* emissions?
**Observation:** Not necessarily. This scatter plot helps visualize the scale vs. intensity.



In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(
    data=df_countries[df_countries["year"] > 1990], x="population", y="co2_per_capita", hue="country", palette=COUNTRY_COLORS
)
plt.title("Population vs CO2 per Capita (Post-1990)")
plt.ylabel("CO2 per Capita (Tonnes)")
plt.xlabel("Population")
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 19. Population Forecast (2025-2028)
Since population is a driver, we also forecast population growth using a simple Polynomial model.



In [ ]:
print("--- Population Forecast (2025-2028) ---")
plt.figure(figsize=(12, 6))
future_years_pop = np.arange(2025, 2029)

for country in countries:
    country_data = df_eda[df_eda["country"] == country].dropna(subset=["population"])
    if len(country_data) > 5:
        X_pop = country_data[["year"]]
        y_pop = country_data["population"]

        poly_pop = PolynomialFeatures(degree=2)
        X_poly_pop = poly_pop.fit_transform(X_pop)

        model_pop = LinearRegression()
        model_pop.fit(X_poly_pop, y_pop)

        future_years_reshaped = future_years_pop.reshape(-1, 1)
        future_poly_pop = poly_pop.transform(future_years_reshaped)
        pred_pop = model_pop.predict(future_poly_pop)

        color = COUNTRY_COLORS.get(country, "gray")
        plt.plot(country_data["year"], country_data["population"] / 1e6, label=f"{country} (Hist)", color=color)
        plt.plot(future_years_pop, pred_pop / 1e6, linestyle="--", color=color)

plt.title("Population Forecast (2025-2028)")
plt.ylabel("Population (Millions)")
plt.xlabel("Year")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


## 20. Fossil Fuel Sources
Which fuel source is the main culprit? Coal, Oil, or Gas?



In [ ]:
fuel_cols = ["coal_co2", "oil_co2", "gas_co2"]
existing_fuel_cols = [c for c in fuel_cols if c in df_eda.columns]

last_year_data = []
for country in countries:
    country_df = df_eda[df_eda["country"] == country].sort_values("year")
    if not country_df.empty:
        valid_row = country_df.dropna(subset=existing_fuel_cols).tail(1)
        if not valid_row.empty:
            last_year_data.append(valid_row)

if last_year_data:
    df_fuel = pd.concat(last_year_data)
    df_fuel["total_fossil"] = df_fuel[existing_fuel_cols].sum(axis=1)
    for col in existing_fuel_cols:
        df_fuel[f"{col}_share"] = (df_fuel[col] / df_fuel["total_fossil"]) * 100

    plot_data = df_fuel.set_index("country")[[f"{c}_share" for c in existing_fuel_cols]]
    plot_data.columns = [c.replace("_co2", "").title() for c in existing_fuel_cols]
    
    plot_data.plot(kind="bar", stacked=True, figsize=(12, 7), colormap="viridis")
    plt.title("Fossil Fuel CO2 Emission Share (%)")
    plt.ylabel("Share (%)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()


## 21. Production vs. Consumption (Carbon Leakage)
*   **Production:** Emissions produced within a country's borders.
*   **Consumption:** Emissions adjusted for trade (Imported goods' emissions count towards the consumer).
*   **Gap:** If Consumption > Production, the country is "outsourcing" pollution.



In [ ]:
for country in countries:
    country_data = df_eda[df_eda["country"] == country].sort_values("year")

    if "consumption_co2" in country_data.columns and not country_data["consumption_co2"].isnull().all():
        plt.figure(figsize=(10, 5))
        sns.lineplot(data=country_data, x="year", y="co2", label="Production (Territorial)", color=COUNTRY_COLORS.get(country))
        sns.lineplot(data=country_data, x="year", y="consumption_co2", label="Consumption (Trade-Adjusted)", color="black", linestyle="--")
        plt.fill_between(country_data["year"], country_data["co2"], country_data["consumption_co2"], alpha=0.1, color="gray")
        plt.title(f"{country}: Production vs Consumption")
        plt.ylabel("CO2 Emissions")
        plt.legend()
        plt.grid(True, linestyle="--", alpha=0.7)
        plt.show()


## 22. Interactive 3D Visualization
Finally, we create an **interactive 3D globe** using Plotly to visualize emissions geographically.
*   **Markers:** '!' indicates pollution levels.
*   **Color Scale:** Green (Low) -> Red (High).



In [ ]:
# Coordinates for 3D Map
COUNTRY_COORDS = {
    'China': {'lat': 35.8617, 'lon': 104.1954},
    'United States': {'lat': 37.0902, 'lon': -95.7129},
    'Germany': {'lat': 51.1657, 'lon': 10.4515},
    'Russia': {'lat': 61.5240, 'lon': 105.3188},
    'Turkey': {'lat': 38.9637, 'lon': 35.2433},
    'India': {'lat': 20.5937, 'lon': 78.9629}
}

def get_pollution_color(co2_value, min_co2=100, max_co2=12000):
    if co2_value <= 0: return 'rgb(100, 200, 100)'
    normalized = min(1, max(0, (co2_value - min_co2) / (max_co2 - min_co2)))
    
    if normalized < 0.25:
        r = int(100 + normalized * 4 * 155)
        g = int(200 + normalized * 4 * 55)
        b = int(100 - normalized * 4 * 100)
    elif normalized < 0.5:
        t = (normalized - 0.25) * 4
        r, g, b = 255, 255, 0
    elif normalized < 0.75:
        t = (normalized - 0.5) * 4
        r, g, b = 255, int(255 - t * 130), 0
    else:
        t = (normalized - 0.75) * 4
        r, g, b = 255, int(125 - t * 125), 0
    
    return f'rgb({r}, {g}, {b})'

def create_3d_globe(df, year):
    countries = list(COUNTRY_COORDS.keys())
    df_year = df[df['year'] == year]
    
    lats, lons, hover_texts, marker_texts, sizes, colors = [], [], [], [], [], []
    
    for country in countries:
        coords = COUNTRY_COORDS[country]
        country_data = df_year[df_year['country'] == country]
        
        co2 = country_data['co2'].values[0] if not country_data.empty else 0
        
        lats.append(coords['lat'])
        lons.append(coords['lon'])
        colors.append(get_pollution_color(co2))
        sizes.append(max(20, min(60, co2 / 150)) if co2 > 0 else 20)
        marker_texts.append("!!!" if co2 > 8000 else "!!" if co2 > 4000 else "!")
        
        hover_texts.append(f"<b>{country}</b><br>CO2: {co2:,.0f} Mt")
    
    fig = go.Figure(go.Scattergeo(
        lon=lons, lat=lats, text=marker_texts, hovertext=hover_texts,
        mode='text+markers',
        marker=dict(size=sizes, color=colors, opacity=0.9, line=dict(width=3, color='white')),
        textfont=dict(size=[s*0.8 for s in sizes], color='white')
    ))
    
    fig.update_geos(
        projection_type="orthographic", showland=True, landcolor='rgb(40, 60, 40)',
        showocean=True, oceancolor='rgb(30, 50, 80)', bgcolor='rgba(0,0,0,0)'
    )
    
    fig.update_layout(
        title=f'3D CO2 Emissions - {year}',
        paper_bgcolor='rgb(20, 20, 30)',
        font=dict(color='white'),
        margin=dict(l=0, r=0, t=50, b=0),
        height=600
    )
    
    fig.show()

print("--- Interactive 3D Globe (2024) ---")
create_3d_globe(df_eda, 2024)
